In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
caminhos = [
    "files/apostila.pdf",
    "files/LLM.pdf",
    ]

paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [3]:
diretorio = 'arquivos/chat_retrieval_db'

embeddings_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

In [4]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

In [5]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

In [6]:
pergunta = "O que é Hugging Face e como faço para acessá-lo?"
chat_chain.invoke({"query": pergunta})

{'query': 'O que é Hugging Face e como faço para acessá-lo?',
 'result': 'Hugging Face é uma comunidade e plataforma que reúne centenas de milhares de modelos de contribuidores, focada em resolver diversos casos de uso em inteligência artificial, como geração de texto, resumo e classificação. Para acessá-lo, você pode visitar o site oficial da Hugging Face, onde você encontrará uma vasta coleção de modelos, além de tutoriais e recursos para ajudá-lo a começar a usar esses modelos em seus projetos.'}

In [7]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
"""Utilize o contexto fornecido para responder a pergunta ao final. 
Se você não sabe a resposta, apenas diga que não sabe e não invente uma resposta.
Utilize três frases no máximo, mantenha a resposta concisa.

Contexto: {context}

Pergunta: {question}

Resposta:
"""
)

In [8]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":chain_prompt},
    return_source_documents=True
)

In [9]:
pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma comunidade que oferece centenas de milhares de modelos de código aberto, incluindo ferramentas para geração de texto e classificação. Para acessar, você pode visitar o site da Hugging Face e explorar os modelos disponíveis ou participar da comunidade. Além disso, há recursos educacionais, como cursos e palestras, para ajudá-lo a se aprofundar no uso de LLMs.


In [10]:
resposta['source_documents']

[Document(metadata={'creator': 'PyPDF', 'source': 'files/LLM.pdf', 'page': 5, 'page_label': '6', 'total_pages': 9, 'doc_id': 71, 'moddate': "D:20231017132635Z00'00'", 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'creationdate': "D:20231017132635Z00'00'"}, page_content='. Comunidades como a Hugging Face reúnem centenas de milhares de modelos de contribuidores que podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. A comunidade de código aberto está rapidamente alcançando o desempenho dos modelos proprietários, mas ainda não conseguiu igualar o desempenho de algo como o GPT-4.'),
 Document(metadata={'creationdate': "D:20231017132635Z00'00'", 'page_label': '8', 'total_pages': 9, 'doc_id': 77, 'moddate': "D:20231017132635Z00'00'", 'page': 7, 'source': 'files/LLM.pdf', 'creator': 'PyPDF', 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext'}, page_content='E agora, o que fazer se eu quiser começar a usar LL

In [11]:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})

set_debug(False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "O que é Hugging Face e como faço para acessá-lo?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "O que é Hugging Face e como faço para acessá-lo?",
  "context": ". Comunidades como a Hugging Face reúnem centenas de milhares de modelos de contribuidores que podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. A comunidade de código aberto está rapidamente alcançando o desempenho dos modelos proprietários, mas ainda não conseguiu igualar o desempenho de algo como o GPT-4.\n\nE agora, o que fazer se eu quiser começar a usar LLMs?    Isso depende de onde você está em sua jornada. Felizmente, temos algumas opções para você. Se você deseja se aprofundar um pouco mais nos LLMs, mas ai

In [12]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma comunidade e plataforma que reúne uma grande variedade de modelos de aprendizado de máquina, especialmente focados em modelos de linguagem, conhecidos como LLMs (Large Language Models). A plataforma é famosa por sua biblioteca "Transformers", que permite fácil acesso e implementação de modelos de aprendizado profundo para tarefas como geração de texto, resumo e classificação. Notavelmente, um dos maiores saltos de desempenho desses modelos veio da integração do feedback humano diretamente no processo de treinamento, o que ajuda a melhorar a qualidade e a relevância das respostas geradas.

Para começar a usar Hugging Face, você pode seguir estes passos:

1. **Visite o site**: Acesse o site oficial do Hugging Face em [huggingface.co](https://huggingface.co).
2. **Criar uma conta**: Inscreva-se gratuitamente para acessar mais recursos e contribuir com a comunidade.
3. **Navegar na biblioteca de modelos**: Explore a vasta coleção de modelos disponíveis, onde você pode en